# Assessment Value Time Slider Choropleth for Downtown Mall

In [1]:
import pandas as pd
import requests
import folium
from folium import plugins
import datetime

In [2]:
#Import .xls retrieved from the GIS Viewer into pandas

#Path to the .xls retrieve from the GIS Viewer
f = r'./data/mall_parcels_no_ends.xls'
#Create a dataframe that reads the .xls file
df = pd.read_excel(f)
#Identify all rows in df where MULTIPIN column is not equal to 1
not_multipin = df['MULTIPIN'] != 1
#Create a new dataframe that only contains the rows identified in not_multipin
df = df[not_multipin]

## Create Query URL's for open portal data based on .xls data

formatted_gpins = [str(x) for x in df['GPIN'].unique()]
formatted_gpins = formatted_gpins
formatted_gpins = ','.join(formatted_gpins)

# Url for Parcel Area Boundary geojson data
parcel_url = f'https://opendata.arcgis.com/datasets/320d465ddf0d498796da6491e21f6dde_43.geojson?where=GPIN%20in%20({formatted_gpins})'
parcel_request = requests.get(parcel_url)
parcel_json = parcel_request.json()

formatted_pins = [f'%27{x}%27' for x in df['PIN'].unique()]
formatted_pins = formatted_pins
formatted_pins = ','.join(formatted_pins)

assessment_url = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear&outSR=4326&f=json"
assessment_request = requests.get(assessment_url)
assessment_json = assessment_request.json()

# Create data series based on features and combine data frames into a single df
assessment_df = pd.DataFrame(assessment_json['features'])
# Create a single data frame based on combined series data
assessments = pd.DataFrame([x for x in assessment_df['attributes']], dtype = 'object')
assessments = assessments.astype({'TaxYear': 'int64', 'ImprovementValue': 'int64','LandValue': 'int64','TotalValue': 'int64', 'ParcelNumber': 'str'})
assessments = assessments.drop(['ImprovementValue', 'LandValue'], axis=1)
#----------------
#Create data frame from df that holds PIN and GPIN.
df_key = pd.DataFrame(df[['PIN','GPIN']], dtype = 'str')
#----------------
# Merge df_key with assessments
d = pd.merge(assessments, df_key, how='inner', left_on=['ParcelNumber'], right_on=['PIN'])

In [3]:
bins = [0, 100000, 250000, 500000, 1000000, 2000000, 5000000, 10000000, 25000000, 50000000]
bin_color = ['#f7fcfd', '#EEF9FB', '#e5f5f9','#ccece6','#99d8c9','#66c2a4','#41ae76','#238b45','#006d2c','#00441b']

t = d.drop(['ParcelNumber','PIN'], axis = 1)
t.head()
lol = t.values.tolist()
lol = [[x[2],f'{x[0]}-1-1', x[1]] for x in lol]
lol
for row in lol:
    if row[2] in range(bins[0],bins[1]+1):
        #row[2] = {'color': bin_color[0]}
        row[2] = bin_color[0]
    elif row[2] in range(bins[1],bins[2]+1):
        #row[2] = {'color': bin_color[1]}
        row[2] = bin_color[1]
    elif row[2] in range(bins[2],bins[3]+1):
        #row[2] = {'color': bin_color[2]}
        row[2] = bin_color[2]
    elif row[2] in range(bins[3],bins[4]+1):
        #row[2] = {'color': bin_color[3]}  
        row[2] = bin_color[3]  
    elif row[2] in range(bins[4],bins[5]+1):
        #row[2] = {'color': bin_color[4]}
        row[2] = bin_color[4]
    elif row[2] in range(bins[5],bins[6]+1):
        #row[2] = {'color': bin_color[5]}
        row[2] = bin_color[5]
    elif row[2] in range(bins[6],bins[7]+1):
        #row[2] = {'color': bin_color[6]}
        row[2] = bin_color[6]
    elif row[2] in range(bins[7],bins[8]+1):
        #row[2] = {'color': bin_color[7]}
        row[2] = bin_color[7]
    elif row[2] in range(bins[8],bins[9]+1):
        #row[2] = {'color': bin_color[8]}
        row[2] = bin_color[8]
    elif row[2] in range(bins[9],bins[10]+1):
        #row[2] = {'color': bin_color[9]}
        row[2] = bin_color[9]

In [4]:
for x in lol:
    date_split = x[1].split('-')
    y = int(date_split[0])
    m = int(date_split[1])
    d = int(date_split[2])
    x[1] = datetime.datetime.timestamp(datetime.datetime(y,m,d))

In [5]:
df_lol = pd.DataFrame(lol, columns = ['GPIN', 'Year', 'color'])
df_lol['GPIN'] = df_lol.astype({'GPIN': 'int'})
df_lol.head()

,GPIN,Year,color
0,6861,1.546319e+09,#41ae76
1,6861,1.514783e+09,#41ae76
2,6861,1.483247e+09,#41ae76
3,6861,1.451624e+09,#66c2a4
4,6861,1.420088e+09,#66c2a4


In [51]:
styledata = {}

for gpin in df['GPIN'].unique():
    dfx = pd.DataFrame(df_lol[df_lol['GPIN'] == gpin])
    dfx = dfx.drop(['GPIN'], axis=1)
    dfx = dfx.set_index(['Year'])

    styledata[gpin] = dfx
    
styledict = {int(GPIN): data.to_dict(orient='index') for GPIN, data in styledata.items()}
styledict.keys()

dict_keys([7309, 14744, 6656, 7426, 7021, 6984, 7197, 6819, 6876, 7447, 7029, 6724, 7511, 7484, 7042, 7005, 6807, 7515, 7543, 6923, 17202, 7128, 7541, 7199, 6805, 7120, 7111, 6877, 7047, 6569, 7053, 6958, 6674, 7082, 7037, 6668, 6930, 7063, 6666, 7213, 6740, 7078, 6897, 7091, 6726, 6966, 7104, 6806, 6869, 6633, 6881, 7272, 7068, 6703, 6947, 6832, 6625, 7141, 6604, 6696, 7162, 6694, 7171, 6848, 7346, 6537, 6662, 6493, 7017, 7330, 7025, 6655, 7033, 6644, 6861, 6898, 7232, 6979, 6918, 7059, 6938, 7249, 7070, 7259, 7087, 7096, 7266, 7103, 7275, 7308, 7163, 7510, 7319, 7329, 7339, 7185, 7348, 7072, 7198, 7358, 7207, 7374, 7390, 7441, 7456, 7471, 7664, 7482, 7717, 7507, 7524, 17098, 17097])

In [52]:
features = []
for x in range(len(df_lol['GPIN'].unique())):
    geometry = parcel_json['features'][x]['geometry']
    properties = parcel_json['features'][x]['properties']
    gpin = properties['GPIN']
    ft = {"gpin": gpin, "type": "Feature", "properties": properties, "geometry": geometry}
    features.append(ft)
    
parcel_geojson_updated = {"type": "FeatureCollection", "features": features}

In [53]:
m = folium.Map(location=[38.03090, -78.48044], zoom_start=17, tiles='OpenStreetMap')

#folium.GeoJson(parcel_json,name='geojson').add_to(m)

folium.plugins.TimeSliderChoropleth(
    data = parcel_geojson_updated,
    styledict=styledict,
    name='Assessment Data',
    overlay=True,
    control=True,
    show=True).add_to(m)

m.save('./TimeSliderChoropleth.html')

m

In [54]:
m = folium.Map(location=[38.03090, -78.48044], zoom_start=17, tiles='OpenStreetMap')

folium.GeoJson(parcel_geojson_updated,name='geojson').add_to(m)

m.save('./TimeSliderChoropleth.html')

m